In [11]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [12]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv(shape= [4, 4])

In [13]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    # Start with a random (all 0) value function and random (all 0) policy
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA]) 
        
    while True:
        delta = 0
        
        # For each state, perform a "full backup"
        for s in range(env.nS):
            max_v = -np.inf
            
            # current best policy
            current_policy = np.argmax(policy[s])
            
            # Look at the possible next actions
            for a, _ in enumerate(policy[s]):
                
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    
                    # Calculate the optimal value (update bellman optimality equation)
                    v = prob * (reward + discount_factor * V[next_state])
                    max_v = max(v, max_v)
                    
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(V[s] - max_v))
            V[s] = max_v
            
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
            
    # Output a deterministic policy
    for s in range(env.nS):
        
        action_states = np.zeros(env.nA)
        for a, _ in enumerate(policy[s]):       
            prob, next_state, reward, done = env.P[s][a][0]
            action_states[a] = prob * (reward + discount_factor * V[next_state])
            
        # greedily select best action
        best_action = np.argmax(action_states)
        
        # always take best action
        policy[s, best_action] = 1.0
            
    return policy, V

In [14]:
# time 
import time
start = time.time()
policy, v = value_iteration(env)
end = time.time()
duration = end - start
print('Value iteration time: {}'.format(duration))

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value iteration time: 0.002599000930786133
Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [15]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)